<a href="https://colab.research.google.com/github/nrcellini/Scripts/blob/master/DEMO_V1_budget_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libaries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

#Input Necessary Company, Budget, & Date Parameters

In [ ]:
#these aspects are hardcoded and must be manually changed whenever creating a new budget calendar
companies = {'A':
                ['Jack', 'Jill'], 
             'B':
                ['Bubbles','Blossom', 'Buttercup'],
             'C':
                ['Total']
             }

budget_versions = {
                   'V1':
                      {'A': 1000000, 
                       'B': 3000000,
                       'C': np.NaN},
                    
                   'V2':
                      {'A': 2000000, 
                       'B': 9000000,
                       'C': 10000000}
                   }

start_date = '2022-01-01'
end_date = '2022-12-31'

weekend = ['Saturday','Sunday']

holidays = ['2022-01-01',
            '2022-01-17',
            '2022-02-21',
            '2022-05-30',
            '2022-06-20',
            '2022-07-04',
            '2022-09-05',
            '2022-10-10',
            '2022-11-11',
            '2022-11-24',
            '2022-12-26'
            ]


#Create Date & Company/Employee Spine

In [ ]:
df = pd.DataFrame({'Date': pd.date_range(start_date, end_date)})

df['Company'] = [companies.keys()]*len(df)
df = df.explode('Company')
df['Employee'] = df.apply(lambda row: companies[row['Company']], axis=1)
df = df.explode('Employee')

df['Day'] = df['Date'].dt.day_name()
df['Quarter'] = df['Date'].dt.quarter
df['Month'] = df['Date'].dt.month
df['Week'] = (df['Date'] - df['Date'].dt.weekday*np.timedelta64(1, 'D')).dt.strftime('%Y%m%d')
df['Year'] = df['Date'].dt.year
df['Date'] = df['Date'].dt.strftime('%Y%m%d')
 
df

,Date,Company,Employee,Day,Quarter,Month,Week,Year
0,20220101,A,Jack,Saturday,1,1,20211227,2022
0,20220101,A,Jill,Saturday,1,1,20211227,2022
0,20220101,B,Bubbles,Saturday,1,1,20211227,2022
0,20220101,B,Blossom,Saturday,1,1,20211227,2022
0,20220101,B,Buttercup,Saturday,1,1,20211227,2022
...,...,...,...,...,...,...,...,...
364,20221231,A,Jill,Saturday,4,12,20221226,2022
364,20221231,B,Bubbles,Saturday,4,12,20221226,2022
364,20221231,B,Blossom,Saturday,4,12,20221226,2022
364,20221231,B,Buttercup,Saturday,4,12,20221226,2022


#Create Holiday and Work Day Dictionaries

In [ ]:
def make_weekend_dict():
  weekend_dict = {'Monday':0,'Tuesday':0, 'Wednesday':0, 'Thursday': 0, 'Friday':0, 'Saturday': 0, 'Sunday': 0}
  for day in weekend :
    weekend_dict.update({day:1})
  return weekend_dict

weekend_dict = make_weekend_dict()

df['is_weekend'] = df.apply(lambda row: weekend_dict[row['Day']], axis=1)

In [ ]:
holidays = pd.Series(dt.strptime(d, '%Y-%m-%d') for d in holidays).dt.strftime('%Y%m%d')

def make_holiday_dict() :
  holiday_dict = {}
  for date in df['Date'] :
    holiday_dict[date]=0
  for day in holidays :
    holiday_dict.update({day:1})
  return holiday_dict

holiday_dict = make_holiday_dict()
df['is_holiday'] = df.apply(lambda row: holiday_dict[row['Date']], axis=1)
df['is_workday'] = np.where((df['is_weekend'] == 1) | (df['is_holiday'] == 1), 0, 1)
df

,Date,Company,Employee,Day,Quarter,Month,Week,Year,is_weekend,is_holiday,is_workday
0,20220101,A,Jack,Saturday,1,1,20211227,2022,1,1,0
0,20220101,A,Jill,Saturday,1,1,20211227,2022,1,1,0
0,20220101,B,Bubbles,Saturday,1,1,20211227,2022,1,1,0
0,20220101,B,Blossom,Saturday,1,1,20211227,2022,1,1,0
0,20220101,B,Buttercup,Saturday,1,1,20211227,2022,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
364,20221231,A,Jill,Saturday,4,12,20221226,2022,1,0,0
364,20221231,B,Bubbles,Saturday,4,12,20221226,2022,1,0,0
364,20221231,B,Blossom,Saturday,4,12,20221226,2022,1,0,0
364,20221231,B,Buttercup,Saturday,4,12,20221226,2022,1,0,0


#Daily Spread

In [ ]:
# # #this is assuming the budget is the same across employees, if this is not true, then need to use a dictionary for each employee instead

In [ ]:
df['daily_multiplier'] = 1/df.groupby(['Company', 'Employee'])['is_workday'].transform('sum')
def iter_over_versions():
  for key,value in budget_versions.items() :
    col_name_yearly = 'yearly_budget_' +str(key)
    col_name_daily = 'daily_budget_'+str(key)
    col_name_weekly = 'weekly_budget_'+str(key)
    col_name_monthly = 'monthly_budget_'+str(key)
    col_name_quarterly = 'quarterly_budget_'+str(key)
  
    df[col_name_yearly] = df.apply(lambda row: value[row['Company']], axis=1)/df.groupby(['Company','Date'])['Employee'].transform('count')
    df[col_name_daily] = (df['daily_multiplier']*df[col_name_yearly]*df['is_workday']).round(2)
    df[col_name_weekly] = df.groupby(['Company','Employee','Week'])[col_name_daily].transform('sum')
    df[col_name_monthly] = df.groupby(['Company','Employee','Month','Year'])[col_name_daily].transform('sum')
    df[col_name_quarterly] = df.groupby(['Company','Employee','Quarter','Year'])[col_name_daily].transform('sum')

    continue
  return df.head(15)

iter_over_versions()

,Date,Company,Employee,Day,Quarter,Month,Week,Year,is_weekend,is_holiday,...,yearly_budget_V1,daily_budget_V1,weekly_budget_V1,monthly_budget_V1,quarterly_budget_V1,yearly_budget_V2,daily_budget_V2,weekly_budget_V2,monthly_budget_V2,quarterly_budget_V2
0,20220101,A,Jack,Saturday,1,1,20211227,2022,1,1,...,500000.0,0.0,0.0,40000.0,124000.0,1000000.0,0.0,0.0,80000.0,248000.0
0,20220101,A,Jill,Saturday,1,1,20211227,2022,1,1,...,500000.0,0.0,0.0,40000.0,124000.0,1000000.0,0.0,0.0,80000.0,248000.0
0,20220101,B,Bubbles,Saturday,1,1,20211227,2022,1,1,...,1000000.0,0.0,0.0,80000.0,248000.0,3000000.0,0.0,0.0,240000.0,744000.0
0,20220101,B,Blossom,Saturday,1,1,20211227,2022,1,1,...,1000000.0,0.0,0.0,80000.0,248000.0,3000000.0,0.0,0.0,240000.0,744000.0
0,20220101,B,Buttercup,Saturday,1,1,20211227,2022,1,1,...,1000000.0,0.0,0.0,80000.0,248000.0,3000000.0,0.0,0.0,240000.0,744000.0
0,20220101,C,Total,Saturday,1,1,20211227,2022,1,1,...,NaN,NaN,0.0,0.0,0.0,10000000.0,0.0,0.0,800000.0,2480000.0
1,20220102,A,Jack,Sunday,1,1,20211227,2022,1,0,...,500000.0,0.0,0.0,40000.0,124000.0,1000000.0,0.0,0.0,80000.0,248000.0
1,20220102,A,Jill,Sunday,1,1,20211227,2022,1,0,...,500000.0,0.0,0.0,40000.0,124000.0,1000000.0,0.0,0.0,80000.0,248000.0
1,20220102,B,Bubbles,Sunday,1,1,20211227,2022,1,0,...,1000000.0,0.0,0.0,80000.0,248000.0,3000000.0,0.0,0.0,240000.0,744000.0
1,20220102,B,Blossom,Sunday,1,1,20211227,2022,1,0,...,1000000.0,0.0,0.0,80000.0,248000.0,3000000.0,0.0,0.0,240000.0,744000.0


#Save df to Excel File

In [ ]:
df.columns

Index(['Date', 'Company', 'Employee', 'Day', 'Quarter', 'Month', 'Week',
       'Year', 'is_weekend', 'is_holiday', 'is_workday', 'daily_multiplier',
       'yearly_budget_V1', 'daily_budget_V1', 'weekly_budget_V1',
       'monthly_budget_V1', 'quarterly_budget_V1', 'yearly_budget_V2',
       'daily_budget_V2', 'weekly_budget_V2', 'monthly_budget_V2',
       'quarterly_budget_V2'],
      dtype='object')

In [ ]:
columns_to_drop = ['Quarter']

In [ ]:
final_budget_df = df.drop(columns_to_drop, axis=1)
final_budget_df.head(15)

,Date,Company,Employee,Day,Month,Week,Year,is_weekend,is_holiday,is_workday,...,yearly_budget_V1,daily_budget_V1,weekly_budget_V1,monthly_budget_V1,quarterly_budget_V1,yearly_budget_V2,daily_budget_V2,weekly_budget_V2,monthly_budget_V2,quarterly_budget_V2
0,20220101,A,Jack,Saturday,1,20211227,2022,1,1,0,...,500000.0,0.0,0.0,40000.0,124000.0,1000000.0,0.0,0.0,80000.0,248000.0
0,20220101,A,Jill,Saturday,1,20211227,2022,1,1,0,...,500000.0,0.0,0.0,40000.0,124000.0,1000000.0,0.0,0.0,80000.0,248000.0
0,20220101,B,Bubbles,Saturday,1,20211227,2022,1,1,0,...,1000000.0,0.0,0.0,80000.0,248000.0,3000000.0,0.0,0.0,240000.0,744000.0
0,20220101,B,Blossom,Saturday,1,20211227,2022,1,1,0,...,1000000.0,0.0,0.0,80000.0,248000.0,3000000.0,0.0,0.0,240000.0,744000.0
0,20220101,B,Buttercup,Saturday,1,20211227,2022,1,1,0,...,1000000.0,0.0,0.0,80000.0,248000.0,3000000.0,0.0,0.0,240000.0,744000.0
0,20220101,C,Total,Saturday,1,20211227,2022,1,1,0,...,NaN,NaN,0.0,0.0,0.0,10000000.0,0.0,0.0,800000.0,2480000.0
1,20220102,A,Jack,Sunday,1,20211227,2022,1,0,0,...,500000.0,0.0,0.0,40000.0,124000.0,1000000.0,0.0,0.0,80000.0,248000.0
1,20220102,A,Jill,Sunday,1,20211227,2022,1,0,0,...,500000.0,0.0,0.0,40000.0,124000.0,1000000.0,0.0,0.0,80000.0,248000.0
1,20220102,B,Bubbles,Sunday,1,20211227,2022,1,0,0,...,1000000.0,0.0,0.0,80000.0,248000.0,3000000.0,0.0,0.0,240000.0,744000.0
1,20220102,B,Blossom,Sunday,1,20211227,2022,1,0,0,...,1000000.0,0.0,0.0,80000.0,248000.0,3000000.0,0.0,0.0,240000.0,744000.0


In [ ]:
file_name = 'budget_calendar_test.xlsx'

In [ ]:
writer = pd.ExcelWriter(file_name)
final_budget_df.to_excel(writer, index = False)
writer.save()
print('{} successfully saved to Excel File.'.format(file_name))

budget_calendar_test.xlsx successfully saved to Excel File.


In [ ]:
from google.colab import files
files.download(file_name)
print('{} successfully downloaded.'.format(file_name))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

budget_calendar_test.xlsx successfully downloaded.
